In [1]:
#Program Name: skim.ipynb 
#Author: Deepesh B. Verma (dbv293)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from optparse import OptionParser
import import_ipynb

from parameters import *
from geometry import *
from wrapper import *

importing Jupyter notebook from parameters.ipynb
importing Jupyter notebook from geometry.ipynb
importing Jupyter notebook from wrapper.ipynb


In [3]:
#Imports necessary inputs parameters and fields from GENE simulation. 

#Gets the necessary parameters to run SKiM model. 
suffix = '_0001' #specifc run that we are investigating
pars = init_read_parameters_file(suffix)

#Gets the magnetic field parameters information for SKiM. 
geom_type, geom_pars, geom_coeff = init_read_geometry_file(suffix,pars)

#Imports frequencies file from GENE. 
om = np.genfromtxt('omega'+suffix)

#Imports the magnetic field from 'magn_geometry'_XXXX file and the coordinates from coord_XXXX file.
geom_h5 = h5py.File(geom_type + suffix + '.h5', 'r')
coord = h5py.File('coord' + suffix + '.h5', 'r')

#Reconstructs the zgrid and the jacobian. 
zgrid, jacobian = reconstruct_zgrid_h5(geom_h5, coord, pars)

#Calculates kperp, omd_curv, omd_gradB, and the magnetic field.
kperp, omd_curv, omd_gradB, Bfield = calc_kperp_omd_h5(coord, geom_h5, geom_type, geom_coeff, pars, center_only = False, output_B = True)



In [4]:
#Imports the eigenfunction phi from the field_XXXX file.
phi, apar = eigenfunctions_from_field_file(pars, suffix, center_only = False)

def normalize(phi0): 
    """ 
    Normalizes the absolute value of the eignefunction phi to 1. 
    """
    phi = phi0.copy()
    abs_phi = abs(phi)
    max = np.argmax(abs_phi)
    phimax = phi[max]
    phi /= phimax

    return phi

#Normalizes the input eigenfunction to 1. 
phi = normalize(phi)

In [5]:
#SKiM Calculation 

#Calculates the eigen-average of kperp, omd, and the bessel factor. 
avg_kperp, avg_omd, avg_bessel_factor = eigenfunction_average_bessel(zgrid, jacobian, kperp, omd_curv, phi)

#Defines fp. 
omn = float(pars['omn1'])
omt = float(pars['omt1'])
gamma = om[1]
fp = omn / (omn + omt)

#Calculates the root of the bessel function using bisection. Serves as an approximation to kperp.
root = bisection_fp(0,avg_kperp,avg_bessel_factor,1.e-4,fp)

#Calculates the eigen-average of omd considering full fp effects.
avg_omd_full, avg_omd_full_j0 = avg_omegad_bessel_full_omn_omt(zgrid, jacobian, kperp, omd_curv, phi, root, omn, omt)

#Calculates the eigen-average of kz considering full fp effects.
zmax = max(zgrid)
zmin = -zmax 
avg_kz = kz_from_dfielddz_bessel_full_omn_omt(root, kperp, zgrid, jacobian, phi, omn, omt, zmin, zmax)

In [6]:
#Outputs Calculation
f = open("skim" + suffix,'w')
f.write('#1.avg_kz 2.avg_kperp 3. avg_omd 4. fp 5. gamma\n')
np.savetxt(f,np.column_stack((avg_kz,avg_kperp,avg_omd_full,fp,gamma)),fmt='%4.4f', delimiter = "  ")
f.close()

In [7]:
#Check to see what is being written to file :/
print(avg_kz,avg_kperp,avg_omd_full,fp,gamma)

0.9553081589335278529 1.6823738789787406336 0.3624953794853734938 0.0 1.621
